In [1]:
import torch
from torchaudio.datasets import SPEECHCOMMANDS
import os
import matplotlib.pyplot as plt
import IPython.display as ipd
import torchaudio
import glob
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.utils.data as tud
import pandas as pd
import torch
import json

/home/hacene/.cache/pypoetry/virtualenvs/speech-keyword-spotting-oTjKaf-s-py3.8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [51]:

class SubsetSC(SPEECHCOMMANDS):
    def __init__(self, subset: str = None):
        super().__init__("../..", download=False)

        def load_list(filename):
            filepath = os.path.join(self._path, filename)
            with open(filepath) as fileobj:
                return [os.path.join(self._path, line.strip()) for line in fileobj]

        if subset == "validation":
            self._walker = load_list("validation_list.txt")
        elif subset == "testing":
            self._walker = load_list("testing_list.txt")
        elif subset == "training":
            excludes = load_list("validation_list.txt") + load_list("testing_list.txt")
            excludes = set(excludes)
            self._walker = [w for w in self._walker if w not in excludes]

In [52]:
# Create training and testing split of the data. We do not use validation in this tutorial.
train_set = SubsetSC("training")
test_set = SubsetSC("testing")

waveform, sample_rate, label, speaker_id, utterance_number = train_set[0]


In [53]:

waveform, sample_rate, label, speaker_id, utterance_number

(tensor([[-0.0658, -0.0709, -0.0753,  ..., -0.0700, -0.0731, -0.0704]]),
 16000,
 'backward',
 '0165e0e8',
 0)

In [55]:
class AudioDataset(tud.Dataset):
    """
    A rapper class for the UrbanSound8K dataset.
    """

    def __init__(
        self,  
        annotation_file, 
        audio_dir, 
        folds,
        target_sample_rate=16000,
        num_samples=32000):
        self.audio_file = pd.read_csv(annotation_file)
        self.folds = folds
        self.audio_paths = glob.glob(audio_dir + '/*' + str(self.folds) + '/*')
        self.num_samples = num_samples
        self.target_sample_rate = target_sample_rate
    
    

    def __len__(self):
        return len(self.audio_paths)

    def __getitem__(self, idx):
        
        audio_sample_path = self._get_audio_sample_path(idx)
        label = self._get_audio_sample_label(audio_sample_path)
        signal, sr = torchaudio.load(audio_sample_path)
        print(signal.shape, sr)
        signal = self._resample_if_necessary(signal, sr)
        print(signal.shape, sr)
        signal = self._mix_down_if_necessary(signal)
        signal = self._cut_if_necessary(signal)
        print(signal.shape, sr)
        signal = self._right_pad_if_necessary(signal)
        print(signal.shape, sr)
        
        return signal, label

    def _cut_if_necessary(self, signal):
        if signal.shape[1] > self.num_samples:
            signal = signal[:, :self.num_samples]
        return signal

    def _right_pad_if_necessary(self, signal):
        length_signal = signal.shape[1]
        if length_signal < self.num_samples:
            num_missing_samples = self.num_samples - length_signal
            last_dim_padding = (0, num_missing_samples)
            signal = torch.nn.functional.pad(signal, last_dim_padding)
        return signal

    def _resample_if_necessary(self, signal, sr):
        if sr != self.target_sample_rate:
            resampler = torchaudio.transforms.Resample(sr, self.target_sample_rate)
            signal = resampler(signal)
        return signal

    def _mix_down_if_necessary(self, signal):
        if signal.shape[0] > 1:
            signal = torch.mean(signal, dim=0, keepdim=True)
        return signal

    def _get_audio_sample_path(self, idx):
        audio_path = self.audio_paths[idx]
        return audio_path

    def _get_audio_sample_label(self, audio_path):
        audio_name = audio_path.split(sep='/')[-1]
        label = self.audio_file.loc[self.audio_file.slice_file_name == audio_name].iloc[0,-2]
        return label

In [56]:
train_ds = AudioDataset(
    audio_dir="/home/hacene/Documents/UrbanSound8K/audio/", 
    annotation_file="/home/hacene/Documents/UrbanSound8K/metadata/UrbanSound8K.csv",
    folds=[1,2, 3, 4, 5, 6, 7, 8, 9],
    )
len(train_ds)

7895

In [57]:
train_ds[0][0].shape

torch.Size([1, 107520]) 96000
torch.Size([1, 17920]) 96000
torch.Size([1, 17920]) 96000
torch.Size([1, 32000]) 96000


torch.Size([1, 32000])

In [59]:
waveform.shape

torch.Size([1, 16000])

In [98]:
mel_spectrogram = torchaudio.transforms.MelSpectrogram(
        sample_rate=16000,
        n_fft=512,
        hop_length=128,
        n_mels=64
    )

In [99]:
spec = mel_spectrogram(waveform)

In [100]:
spec.shape

torch.Size([1, 64, 126])

In [119]:
def make_layers():
    layers = []
    in_channels = 1
    for v in [64, "M", 128, "M", 256, 256, "M", 512, 512]:
        if v == "M":
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    layers += [nn.AvgPool2d(kernel_size=(2, 4), stride=(2, 2))]
    return nn.Sequential(*layers)

In [120]:
vgg = make_layers()

In [121]:
y = vgg(spec.unsqueeze(0))
print(y.shape)

torch.Size([1, 512, 4, 6])


In [2]:
from keyword_detector.models.transformers import HubertModel, LightHubertModel

In [3]:
model = HubertModel(num_classes=35)

In [4]:
x = torch.randn(64, 32000)

In [5]:
y = model(x)
print("y.shape:", y.shape)

y.shape: torch.Size([64, 1])


In [2]:
from transformers import Wav2Vec2ForSequenceClassification

In [3]:
model = Wav2Vec2ForSequenceClassification.from_pretrained("facebook/wav2vec2-base-960h", num_labels=35)

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForSequenceClassification: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['classifier.weight', 'classifier.bias', 'projector.weight', 'wav2vec2.masked_spec_embed', 'projector.bias']
You should probably TRAIN this model on a down-stream task to be 

In [10]:
model.wav2vec2;

In [11]:
model.classifier

Linear(in_features=256, out_features=35, bias=True)

In [12]:
model.projector

Linear(in_features=768, out_features=256, bias=True)

In [5]:
from transformers import (
    HubertForSequenceClassification,
    Wav2Vec2ForSequenceClassification,
    Wav2Vec2FeatureExtractor,
)
import torch.nn as nn
import torch


class Wav2Vec2AudioModel(nn.Module):
    """Define the Wav2Vec2 model. It is a pre-trained model from the HuggingFace library.
    The model is a sequence classification model, which means that it takes a
    1-second-long audio array (sampled at sample_rate=1600 in mono) and outputs the
    logits of an audio keyword. The vector can then passed through a softmax function
    to get the probability distribution over the classes.

    Parameters
    ----------
    num_classes : int
        Number of classes in the dataset. Default is 35.
    """

    def __init__(self, num_classes: int = 35, sampling_rate: int = 16000):
        super().__init__()
        self.num_classes = num_classes
        self.feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(
            "superb/wav2vec2-base-superb-ks"
        )
        self.wav2vec2_model = Wav2Vec2ForSequenceClassification.from_pretrained(
            "superb/wav2vec2-base-superb-ks",
            num_labels=self.num_classes,
            ignore_mismatched_sizes=True,
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Forward pass of the model."""
        x = x.squeeze()
        inputs = self.feature_extractor(x, return_tensors="pt", sampling_rate=16000)
        return self.wav2vec2_model(**inputs).logits

In [6]:
model = Wav2Vec2AudioModel()


/raid/home/labuserterbouche/.cache/pypoetry/virtualenvs/speech-keyword-spotting-mU_yhwjc-py3.8/lib/python3.8/site-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at superb/wav2vec2-base-superb-ks and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([12, 256]) in the checkpoint and torch.Size([35, 256]) in the model instantiated
- classifier.bias: found shape torch.Size([12]) in the checkpoint and torch.Size([35]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infer

In [7]:
x = torch.randn(64, 16000)
y = model(x)
print("y.shape:", y.shape)

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


RuntimeError: Expected 3-dimensional input for 3-dimensional weight [512, 1, 10], but got 4-dimensional input of size [1, 1, 64, 16000] instead